# Starter Code

In [8]:
import numpy as np
import pandas as pd
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report
from joblib import dump, load

In [9]:
df_2018 = pd.read_csv('../data/2018.csv')
df_2017 = pd.read_csv('../data/2017.csv')
df_2016 = pd.read_csv('../data/2016.csv')

def get_classifier_convert(df):
    conditions = [
        (df['fourth_down_converted'] == 1.0),
        (df['fourth_down_failed'] == 1.0),
        (df['field_goal_attempt'] == 1.0),
        (df['punt_attempt'] == 1.0)
    ]

    results = ['CONVERTED', 'FAILED', 'FIELD_GOAL', 'PUNT']

    y = np.select(conditions, results)
    return y

def get_classifier_attempt(df):
    conditions = [
        (df['fourth_down_converted'] == 1.0) | (df['fourth_down_failed'] == 1.0),
        (df['field_goal_attempt'] == 1.0),
        (df['punt_attempt'] == 1.0)
    ]

    results = ['ATTEMPTED', 'FIELD_GOAL', 'PUNT']

    y = np.select(conditions, results)
    return y

def ready_data_convert(df):
    df = df.dropna()
    y = get_classifier_convert(df)
    df = df.drop(columns=['posteam', 'fourth_down_converted', 'fourth_down_failed', 'field_goal_attempt', 'punt_attempt', 'game_date', 'down'])
    return df, y

def ready_data_attempt(df):
    df = df.dropna()
    y = get_classifier_attempt(df)
    df = df.drop(columns=['posteam', 'fourth_down_converted', 'fourth_down_failed', 'field_goal_attempt', 'punt_attempt', 'game_date', 'down'])
    return df, y

# Tree

## Convert 4th down

### Default params

In [10]:
x, y = ready_data_convert(pd.concat([df_2016, df_2017, df_2018]))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

clf = RandomForestClassifier(random_state=0)

clf.fit(X_train, y_train)

dump(clf, '../models/random_forest/convert.joblib')

print(f"Test Accuracy: {clf.score(X_test, y_test)}")
print(f"Train Accuracy: {clf.score(X_train, y_train)}")

report = classification_report(y_test, clf.predict(X_test), zero_division=0)
print(report)

Test Accuracy: 0.8567764001710133
Train Accuracy: 1.0
              precision    recall  f1-score   support

           0       0.24      0.07      0.11        85
   CONVERTED       0.47      0.38      0.42       160
      FAILED       0.47      0.40      0.43       154
  FIELD_GOAL       0.88      0.94      0.91       542
        PUNT       0.92      0.98      0.95      1398

    accuracy                           0.86      2339
   macro avg       0.60      0.55      0.56      2339
weighted avg       0.83      0.86      0.84      2339



### Cross Validation

In [11]:
x, y = ready_data_convert(pd.concat([df_2016, df_2017, df_2018]))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

parameters = {
    'max_features': range(1, math.floor(math.sqrt(len(X_train.iloc[0])))), 
    'max_depth': range(1, 100),
    'n_estimators': range(10, 200),
    'min_samples_split': range(2, 20)
}
clf = RandomizedSearchCV(RandomForestClassifier(random_state=0), parameters, random_state=0, n_iter=200)

clf.fit(X_train, y_train)

dump(clf.best_estimator_, '../models/random_forest/convert_cv.joblib')

print(clf.best_params_)
print(f"Test Accuracy: {clf.score(X_test, y_test)}")
print(f"Train Accuracy: {clf.score(X_train, y_train)}")

report = classification_report(y_test, clf.predict(X_test), zero_division=0)
print(report)

{'n_estimators': 123, 'min_samples_split': 8, 'max_features': 1, 'max_depth': 36}
Test Accuracy: 0.8623343309106456
Train Accuracy: 0.9214239897370109
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        85
   CONVERTED       0.51      0.40      0.45       160
      FAILED       0.49      0.38      0.43       154
  FIELD_GOAL       0.88      0.95      0.91       542
        PUNT       0.92      0.99      0.95      1398

    accuracy                           0.86      2339
   macro avg       0.56      0.54      0.55      2339
weighted avg       0.82      0.86      0.84      2339



## Attempt 4th down

### Default Params

In [12]:
x, y = ready_data_attempt(pd.concat([df_2016, df_2017, df_2018]))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)

dump(clf, '../models/random_forest/attempt.joblib')

print(f"Test Accuracy: {clf.score(X_test, y_test)}")
print(f"Train Accuracy: {clf.score(X_train, y_train)}")

report = classification_report(y_test, clf.predict(X_test), zero_division=0)
print(report)

Test Accuracy: 0.8956819153484396
Train Accuracy: 1.0
              precision    recall  f1-score   support

           0       0.28      0.06      0.10        85
   ATTEMPTED       0.76      0.71      0.74       314
  FIELD_GOAL       0.90      0.93      0.91       542
        PUNT       0.93      0.98      0.95      1398

    accuracy                           0.90      2339
   macro avg       0.72      0.67      0.67      2339
weighted avg       0.88      0.90      0.88      2339



### Cross Validation

In [13]:
x, y = ready_data_attempt(pd.concat([df_2016, df_2017, df_2018]))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

parameters = {
    'max_features': range(1, math.floor(math.sqrt(len(X_train.iloc[0])))), 
    'max_depth': range(1, 100),
    'n_estimators': range(10, 200),
    'min_samples_split': range(2, 20)
}
clf = RandomizedSearchCV(RandomForestClassifier(random_state=0), parameters, random_state=0, n_iter=200)

clf.fit(X_train, y_train)

dump(clf.best_estimator_, '../models/random_forest/attempt_cv.joblib')

print(clf.best_params_)
print(f"Test Accuracy: {clf.score(X_test, y_test)}")
print(f"Train Accuracy: {clf.score(X_train, y_train)}")

report = classification_report(y_test, clf.predict(X_test), zero_division=0)
print(report)

{'n_estimators': 80, 'min_samples_split': 9, 'max_features': 1, 'max_depth': 49}
Test Accuracy: 0.8961094484822574
Train Accuracy: 0.9222792388283088
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        85
   ATTEMPTED       0.77      0.72      0.75       314
  FIELD_GOAL       0.89      0.92      0.91       542
        PUNT       0.92      0.98      0.95      1398

    accuracy                           0.90      2339
   macro avg       0.65      0.66      0.65      2339
weighted avg       0.86      0.90      0.88      2339

